### Wstęp
Poniższy notebook jest kolejnym przykładem użycia polskiego modelu językowego "Bielik" do praktycznych zastosowań. Celem notebooka jest pokazanie przykładowego kodu w języku Python (3.10), który rozwiązuje konkretny problem i jest łatwy do zrozumienia i odtworzenia dzięki opisowi kodu.

Poniższy use case dotyczy poprawy tekstu napisanego w języku polskim. Użyjemy modelu Bielik, aby pomógł nam poprawić tekst elminując z niego różnego rodzaju błędy (m.in. ortograficzne i interpunkcyjne) oraz zmieniając styl wypowiedzi napisanego tekstu na wybrany przez użytkownika.

Zapraszam!

### 1. Pobieranie pakietów

Zaczniemy od pobrania pakietów. Jeżeli ich nie mamy, trzeba je zainstalować poprzez <code>pip</code> lub <code>conda</code> w zależności od naszego w jaki sposób Python został zainstalowany na naszym sprzęcie.
Do pracy będzie na również potrzebna "CUDA", która należy pobrać i zainstalować z oficjalnej strony nvidi [używając tego linku](https://developer.nvidia.com/cuda-downloads).

Następnie należy zainstalować odpowiednia wersje pakietu 'torch' z 'cuda' - polecam wersje 2.3.1 której używa (2.4 wyrzuca czasami problemy szczególnie na Windowsie) z wersja cuda 12.1 - można to zrobić przykładowo przy użyciu takiej komendy
<code>conda install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia</code>

Na końcu dla zwiększenia czytelności ignorujemy ostrzeżenia przy wczytywaniu pakietów

In [1]:
import torch  
import warnings
import re

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

warnings.filterwarnings("ignore")

c:\Users\micbo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Pobieranie Bielika

Zanim przejdziemy do ustawienia naszego Bielika sprawdźmy czy nasze GPU jest rozpoznawane przez torch.
Po tym etapie wpisujemy kilka parametrów do konfiguracji: 
* <code>device</code>, które na stałe zostawiamy jako <code>cuda</code>,
* <code>model_name</code>, którym jest nazwa modelu z HuggingFace, modele Bielik niezależnie od wersji zawsze zaczynają się od <code>speakleash/</code> a następnie wklejamy nazwę modelu. W razie wątpliwości co do nazw i dostępnych modeli, polecam odwiedzić [Speakleash](https://huggingface.co/speakleash) na stronie Hugging Face,
* <code>max_tokens</code> czyli maksymalna długość odpowiedzi liczona w tokenach,
* <code>temperature, top_p, top_k</code> parametry związane z kreatywnością modelu. Więcej o parametrach można przeczytać [tutaj](https://aviralrma.medium.com/understanding-llm-parameters-c2db4b07f0ee).

Pozostała część poniższego kodu wczytuje i ustawiam nam model - korzystamy z ładowania w wersji 4bit aby model był lżejszy i odpowiadał szybciej przy jednoczesnej małej stracie na jakości odpowiedzi (jeżeli mamy dobry sprzęt można pokusić się o lepszą wersję).

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 2060


In [3]:
device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'

max_tokens = 5000

temperature = 0
top_k = 200
top_p = 0


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=quantization_config,
                                            low_cpu_mem_usage = True
                                            )

model.generation_config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards: 100%|██████████| 3/3 [00:48<00:00, 16.30s/it]


### 3. Testowanie działania
Model wczytany, więc zanim przejdziemy do naszego zadania poprawy tekstu napiszmy funkcję <code>generate</code> która będzie udzielała odpowiedzi z Belika na zadany prompt. W ten sposób sprawdzimy czy wszystko działa poprawnie.

In [4]:
def generate(prompt, system = None):

  messages = []

  if system:
    messages.append( {"role": "system", "content": system})

  messages.append( {"role": "user", "content": prompt})


  tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

  if torch.cuda.is_available():
    model_input_ids = tokenizer_output.input_ids.to(device)

    model_attention_mask = tokenizer_output.attention_mask.to(device)

  else:
    model_input_ids = tokenizer_output.input_ids
    model_attention_mask = tokenizer_output.attention_mask

  outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=max_tokens,
                           do_sample=True if temperature else False,
                           temperature = temperature,
                           top_k = top_k,
                           top_p = top_p)
  
  answer = tokenizer.batch_decode(outputs, skip_special_tokens = True)
  answer = re.sub('\["[INST].*$', answer)

  return answer

In [5]:
prompt = "Kim jesteś?"
system = "Jesteś chatboem udzielający odpowiedzi na pytania w języku polskim"
output = generate(prompt, system)

 Jestem chatbotem, wirtualnym asystentem stworzonym do udzielania odpowiedzi na pytania w języku polskim. Moim celem jest pomoc i dostarczanie informacji na różne tematy.


TypeError: sub() missing 1 required positional argument: 'string'

### 4. Funkcja na poprawe tekstu
Przejdźmy do sedna, czyli jak poprawiać tekst. Napiszemy kompleksową funkcję, która na wejściu będzie miała tekst podany przez użytkownika, który wymaga poprawy a na wyjściu poprawiony tekst.

Zaczynamy od definicji funkcji, która będzie posiadała 3 parametry: tekst (w postaci <code>string</code>), listę w której użytkownik może wskazać pod jakim względem chce poprawić tekst oraz 'one_shot' w którym wzmocnimy nasz prompt przykładem prawidłowo wykonanego zadania (co powinno zwiększyć jakość naszej odpowiedzi).

Zaczynamy od zdefiniowania systemu, gdzie nasz model ma się wcielić w rolę eksperta do edycji polskiego tekstu. Następnie rozpoczynamy nasz prompt od jasnej definicji zadania po której następuje dodanie komponentów którymi są kategorie błędów (patrz: źródła 3). Decydujemy się na 7 popularnych kategorii, które są związane z formą pisemną polskiego języka czyli "literowka", "ortografia", "interpunkcja", "fleksja", "skladnia", "leksykalne", "stylistyka". Następnie do promptu dokładamy przykład z poprawą zdania, demonstrując modelowi z jakimi problemami może się zmagać i do jakiego rezultatu powinien w tym przypadku dojść a na koniec dodajemy zabezpieczenie przed zmianą treści (nie chcemy, żeby model zmieniał sens wypowiedzi, a jedynie znajdował i rozwiązywał problemy językowe z nim związane). Na samym końcu wskazujemy mu treść tekstu który ma poprawić.

Użytkownik może wybrać od 1 do 7 kategorii błędów, a domyślną opcję zostawiamy na "wszystko", w której model ma sprawdzić tekst pod każdym z 7 rodzajów błędów.

Mając gotowy cały prompt przechodzimy do generowania odpowiedzi bazując na przygotowanej już wcześniej funkcji <code>generate</code>. Jako, że chcemy aby model nie zmieniał treści a jedynie ją poprawiał, kreatywność jest bardziej naszym przeciwnikiem niż sprzymierzeńcem w związku z czym ustawiamy wartości wszystkich 3 parametrów <code>temperature</code>, <code>top_p</code> oraz <code>top_k</code> na 0. Jako, że na wejściu oczekujemy poprawionego tekstu to deklarujemy wartość <code>max_tokens</code> na 5000, aby nie został on ucięty w trakcie generowania (jeśli mamy dłuższe teksty ustaw wartość na 10 000 lub więcej).

In [ ]:
def resolve_mistakes(text: str, opcje = ['wszystko'], one_shot = True):
    system = "Jesteś ekspertem zajmującym się poprawą tesktów w języku polskim."
    
    prompt_poczatek = "Sprawdź poniższy tekst w języku polskim i popraw go. Zwróć szczególną uwagę na:\n\n"
    
    # Mozliwe opcje do wpisania w drugim argumencie: "literowka", "ortografia", "interpunkcja", "fleksja", "skladnia", "leksykalne", "stylistyka"
    prompt_literowka = "* Literówki (pominięcie, powtórzenie lub wstawienie niewłaściwego znaku)\n\n"
    prompt_ortograficzny = "* Błędy ortograficzne (błędny zapis liter lub ich połączeń, błędna pisownia małą lub wielką literą, błędne stosowanie kropki w skrótach, błędy w pisowni łącznej)\n\n"
    prompt_interpunkcyjny = "* Błędy interpunkcyjne (zbędne użycie znaku interpunkcyjnego, brak znaku interpunkcyjnego, błędne zapisy w przypadku zbiegu znaków interpunkcyjnych, użycie niewłaściwego znaku interpunkcyjnego)\n\n"
    prompt_fleksyjny = "* Błędy fleksyjne (związane z wyborem niewłaściwej formy wyrazu, błędną odmianą lub z błędnym brakiem odmiany)\n\n"
    prompt_skladniowe = "* Błędy składniowe (używanie błędnych form lub konstrukcji w miejscach, których uzupełnienia wymagają wyrazy nadrzędne, błędy w szyku, błędne użycie imiesłowowych równoważników zdań)\n\n"
    prompt_leksykalne = "* Błędy leksykalne (błędne użycie słowa w danej konstrukcji, mylenie wyrazów podobnych brzmieniowo, używanie zbędnych zapożyczeń lub nadużywanie wyrazów obcych, błędy słowotwórcze i powstałe w wyniku skojarzeń słowotwórczych, zniekształcenia związków frazeologicznych)\n\n"
    prompt_stylistyczne = "* Błędy stylistyczne (niedostosowanie form językowych do charakteru i funkcji korespondencji, niezamierzone uczynienie tekstu dwuznacznym)\n\n"
    
    prompt_one_shot = "Poniżej masz przykład: \n\nTeskt do popawy: 'janusz siedział skólnoy za stołm i milczał .'\n\nPrawdiłowy poprawiony teskt: 'Janusz siedział skulony za stołem i milczał.'"
    promt_koniec = "Pamietaj, aby ingerencja w treść była związana tylko z poprawą błędu i nie ingerowala w sens pierwotnej treści. Poniżej tekst do poprawy:\n\n"

    if opcje[0] == 'wszystko':
        opcje = ['literowka', 'ortografia', 'interpunkcja', 'fleksja', 'skladnia', 'leksykalne', 'stylistyka']
    
    prompt_calosc = prompt_poczatek + (prompt_literowka if 'literowka' in opcje else '') + (prompt_ortograficzny if 'ortografia' in opcje else '') + (prompt_interpunkcyjny if 'interpunkcja' in opcje else '') + (prompt_fleksyjny if 'fleksja' in opcje else '') + (prompt_skladniowe if 'skladnia' in opcje else '') + (prompt_leksykalne if 'leksykalne' in opcje else '') + (prompt_stylistyczne if 'stylistyka' in opcje else '') + (prompt_one_shot if one_shot else '') + promt_koniec + text
    
    messages = []
    messages.append( {"role": "system", "content": system})
    messages.append( {"role": "user", "content": prompt_calosc})

    tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

    if torch.cuda.is_available():
      model_input_ids = tokenizer_output.input_ids.to(device)

      model_attention_mask = tokenizer_output.attention_mask.to(device)

    else:
      model_input_ids = tokenizer_output.input_ids
      model_attention_mask = tokenizer_output.attention_mask

    outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=5000,
                           do_sample=True if temperature else False,
                           temperature = 0,
                           top_k = 0,
                           top_p = 0)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    answer = re.sub('\["[INST].*$', answer)

    return answer

### 5. Funkcje na zmiane stylu
Funkcja do zadania poprawy błędów gotowa, czas więc na drugą funkcję która pozwoli nam zmieniać styl wypowiedzi. Wejście i wyjście z naszej funkcji jest podobne jak w poprzednim przypadku: na wejściu oczekujemy tekstu do poprawy a na wyjściu poprawiony tekst. Dodatkowo od użytkownika wymagamy podania jeszcze jednego argumentu, czyli stylu. Użytkownik będzie mógł wybrać jeden z pięciu stylów: "potoczny", "urzedowy", "publicystyczny", "naukowy" i "artystyczny". W tym przypadku nie decydujemy się na deklaracje domyślnej wartości. 

Ustawiamy system jako eksperta od zmiany stylu i rozpoczynamy prompt od przedstawienia zadania oraz wyboru stylu (tym razem użytkownik może wybrać tylko 1 z 5 dostępnych). Na koniec ponownie zabezpieczenie przed zmianą treści oraz "doklejenie" treści tekstu do poprawy. Generowanie odpowiedzi odbywa się w taki sam sposób jak w porzedniej funkcji, jednak tym razem zwiększamy nieco wartości <code>temperature</code>, <code>top_p</code> oraz <code>top_k</code> na nieco większe, gdyż to zadanie wymaga pewnej kreatywności (jednak zbyt duża wartość grozi zmianą sensu wypowiedzi, więc przyjmujemy ostrożne wartości).

In [ ]:
def change_style(text: str, styl: str):
    
    system = "Jesteś ekspertem zajmującym się zmianą stylu tekstu w języku polskim."
    
    prompt_poczatek = "Zmień styl napisanego tekstu na "
    
    # Mozliwe opcje do wpisania w drugim argumencie: "potoczny", "urzedowy", "publicystyczny", "naukowy", "artystyczny"
    prompt_potoczny = "potoczny. "
    prompt_urzedowy = "urzędowy. "
    prompt_publicystyczny = "publicystyczny. "
    prompt_naukowy = "naukowy. "
    prompt_artystyczny = "artystyczny. "

    promt_koniec = "\n\nPamietaj, aby ingerencja w treść była związana tylko ze zmianą i nie ingerowala w sens pierwotnej treści. Poniżej tekst do do zmiany stylu:\n\n"

    prompt_calosc = prompt_poczatek + (prompt_potoczny if styl == 'potoczny' else '') + (prompt_urzedowy if styl == 'urzedowy' else '') + (prompt_publicystyczny if styl == 'publiczystyczny' else '') + (prompt_naukowy if styl == 'naukowy' else '') + (prompt_artystyczny if styl == 'artystyczny' else '') + promt_koniec + text
    
    messages = []
    messages.append( {"role": "system", "content": system})
    messages.append( {"role": "user", "content": prompt_calosc})

    tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

    if torch.cuda.is_available():
      model_input_ids = tokenizer_output.input_ids.to(device)

      model_attention_mask = tokenizer_output.attention_mask.to(device)

    else:
      model_input_ids = tokenizer_output.input_ids
      model_attention_mask = tokenizer_output.attention_mask

    outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=5000,
                           do_sample=True if temperature else False,
                           temperature = 0.1,
                           top_k = 5,
                           top_p = 0.1)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    answer = re.sub('\["[INST].*$', answer)

    return answer

## 6. Przykład 1 - e-mail 
Tekstów napisanych z błędami nie brakuje - z pewnością ten notebook również je zawiera, taka nasza natura, nie wszystko jesteśmy w stanie za każdym razem robić perfekcyjnie. Klasyczny przykładem z życia codziennego (zarówno służbowego jak i prywatnego) są wiadomości email. Kontaktowałem się ostatnio z recepcją swojego dentysty gdyż zauważyłem że nie ma mojego adresu na fakturze, który jest mi potrzebny. Jadąc w tramwaju przygotowałem więc niedbale w notatniku e-mail o następującej treści (niektóre dane zanonimizowane i dołożyłem dodatkowe błędy na potrzeby demonstracji).

Może go zobaczyć poniżej a zawiera on po kolei następujące błędy:
* Spacja przed przecinkiem (błąd interpunkcyjny),
* Rozpoczęcia zdania od małej litery (błąd ortograficzny),
* Pominięte "e" w "zamiszkania" (literówka),
* "proźba" (błąd ortograficzny),
* Błędna odmiana słowa "faktura" (błąd fleksyjny)

Użyjemy naszej funkcji na poprawę tekstu z domyślną opcją "wszystko" oraz spróbujemy włączyć argument "one_shot" licząc na jak najlepszą odpowiedź.

In [ ]:
email = """
Dzień dobry,
Po wtorkowej wizycie otrzymałem fakturę, jednak potrzebuje małej korekty. 
potrzebuje aby w danych nabywcy był dodatkowo adres zamieszkania.

Mój adres zamiszkania (jeżeli kiedyś podawałem , to prawdopodobnie jest nieaktualny bo go niedawno zmieniłem, więc proźba o użycie tego):

ul. Marszałkowska 11C/24
00-001 Warszawa

W załączniku skan aktualnej faktura.

Pozdrawiam
Adam Kowalski
"""

In [ ]:
resolve_mistakes(email, opcje = ['wszystko'], one_shot = True)

 Dzień dobry,
Po wtorkowej wizycie otrzymałem fakturę, jednak potrzebuję drobnej korekty. 
Proszę dodać dodatkowe dane nabywcy: adres zamieszkania.

Mój adres zamieszkania (jeśli kiedyś podawałem, to prawdopodobnie jest nieaktualny, bo go niedawno zmieniłem, więc proszę o użycie tego):

ul. Marszałkowska 11C/24
00-001 Warszawa

W załączniku skan aktualnej faktury.

Pozdrawiam
Adam Kowalski


["[INST] <<SYS>>\nJesteś ekspertem zajmującym się poprawą tesktów w języku polskim.\n<</SYS>>\n\nSprawdź poniższy tekst w języku polskim i popraw go. Zwróć szczególną uwagę na:\n\n* Literówki (pominięcie, powtórzenie lub wstawienie niewłaściwego znaku)\n\n* Błędy ortograficzne (błędny zapis liter lub ich połączeń, błędna pisownia małą lub wielką literą, błędne stosowanie kropki w skrótach, błędy w pisowni łącznej)\n\n* Błędy interpunkcyjne (zbędne użycie znaku interpunkcyjnego, brak znaku interpunkcyjnego, błędne zapisy w przypadku zbiegu znaków interpunkcyjnych, użycie niewłaściwego znaku interpunkcyjnego)\n\n* Błędy fleksyjne (związane z wyborem niewłaściwej formy wyrazu, błędną odmianą lub z błędnym brakiem odmiany)\n\n* Błędy składniowe (używanie błędnych form lub konstrukcji w miejscach, których uzupełnienia wymagają wyrazy nadrzędne, błędy w szyku, błędne użycie imiesłowowych równoważników zdań)\n\n* Błędy leksykalne (błędne użycie słowa w danej konstrukcji, mylenie wyrazów podob

[Tu wnioski to wyniku z Bielik2]

In [ ]:
change_style(email, styl = 'urzedowy')

 Dzień dobry,
W nawiązaniu do otrzymanej faktury z dnia [data] pragnę zwrócić uwagę na konieczność naniesienia drobnej korekty w danych nabywcy. Proszę o uwzględnienie dodatkowego adresu zamieszkania w celu usprawnienia procesu.

Mój adres zamieszkania (jeśli kiedykolwiek podawałem go, to prawdopodobnie jest już nieaktualny, ponieważ niedawno go zmieniłem):

ul. Marszałkowska 11C/24
00-001 Warszawa

W załączeniu przesyłam skan aktualnej faktury.

Z poważaniem,
Adam Kowalski


['[INST] <<SYS>>\nJesteś ekspertem zajmującym się zmianą stylu tekstu w języku polskim.\n<</SYS>>\n\nZmień styl napisanego tekstu na urzędowy. \n\nPamietaj, aby ingerencja w treść była związana tylko ze zmianą i nie ingerowala w sens pierwotnej treści. Poniżej tekst do do zmiany stylu:\n\n\nDzień dobry,\nPo wtorkowej wizycie otrzymałem fakturę, jednak potrzebuje małej korekty. \npotrzebuje aby w danych nabywcy był dodatkowo adres zamieszkania.\n\nMój adres zamiszkania (jeżeli kiedyś podawałem , to prawdopodobnie jest nieaktualny bo go niedawno zmieniłem, więc proźba o użycie tego):\n\nul. Marszałkowska 11C/24\n00-001 Warszawa\n\nW załączniku skan aktualnej faktura.\n\nPozdrawiam\nAdam Kowalski [/INST]  Dzień dobry,\nW nawiązaniu do otrzymanej faktury z dnia [data] pragnę zwrócić uwagę na konieczność naniesienia drobnej korekty w danych nabywcy. Proszę o uwzględnienie dodatkowego adresu zamieszkania w celu usprawnienia procesu.\n\nMój adres zamieszkania (jeśli kiedykolwiek podawałem go,

[Tu wnioski to wyniku z Bielik2]

### 7. Przykład 2 - dokumentacja kodu
Drugi przykład to popularne README będące dokumentacją kodu programistycznego. Nie raz zdarzania się pisać taką dokumentacje niedbalne, gdyż goni nas czas, więc jej jakość może nie być najwyższa, a z naszymi stworzonymi przed chwilą funkcjami możemy ją bardzo szybko podnieść.

Poniżej mamy kawałek README z [repozutorium biblioteki PyTorch](https://github.com/pytorch/pytorch/blob/main/README.md) z którego wcyięto kilka fragmentów i przetłumaczono na język polski przy użyciu Google Translate. Sztucznie dodano również kilka błędów (ta dokumentacja jest akurat dobrze przygotowana, a tłumaczenie nie wygenerowało zbyt wielu błędów):
* Niepotrzebny przecinek po słowie "pełni" (błąd interpunkcyjny),
* "nie zależnie" (błąd ortograficzny),
* Niepotrzebna spacja przed ":" (literówka),
* Przestawienie szyku liter w słowe "zogdny" (literówka),
* Nieuzasadnione powtórzenie "obsługuje" dwa razy z rzędu (błąd stylistyczny).

Zaczniemy od poprawy błędów, ale tym razem przyjmiemy inne założenie - jako użytkownik chce, aby model sprawdził mi jedynie literówki i błędy interpunkcyjne, dlatego w funkcji w argumencie <code>opcje</code> definiuje dwuelementową listę z wymienionymi opcjami błędów.

In [ ]:
dokumentacja = """
**Instalacja**
Polecenia do instalacji binariów za pomocą Conda lub pip wheels znajdują się na naszej stronie internetowej: https://pytorch.org/get-started/locally/

Jeśli instalujesz ze źródła, będziesz potrzebować:

* Python 3.8 lub nowszy (dla Linuksa wymagany jest Python 3.8.1+)
* Kompilator w pełni, obsługujący C++17, taki jak clang lub gcc (wymagany jest gcc 9.4.0 lub nowszy)
* Zdecydowanie zalecamy zainstalowanie środowiska Anaconda. Otrzymasz wysokiej jakości bibliotekę BLAS (MKL) i kontrolowane wersje zależności nie zależnie od dystrybucji Linuksa.

**Obsługa NVIDIA CUDA**
Jeśli chcesz kompilować z obsługą CUDA, wybierz obsługiwaną wersję CUDA z naszej macierzy obsługi, a następnie zainstaluj następujące elementy:

* NVIDIA CUDA
* NVIDIA cuDNN v8.5 lub nowsza
* Kompilator zogdny z CUDA
Uwaga : Możesz zapoznać się z macierzą obsługi cuDNN dla wersji cuDNN z różnymi obsługiwanymi CUDA, sterownikiem CUDA i sprzętem NVIDIA

Jeśli chcesz wyłączyć obsługę CUDA, wyeksportuj zmienną środowiskową USE_CUDA=0. Inne potencjalnie przydatne zmienne środowiskowe można znaleźć w pliku setup.py.

Należy pamiętać, że począwszy od PyTorch 2.5, kompilacja PyTorch z XPU obsługuje zarówno nowe, jak i obsługuje stare ABI C++. Wcześniej XPU obsługiwało tylko nowe ABI C++. Jeśli chcesz kompilować z obsługą Intel GPU, postępuj zgodnie z Intel GPU Support.
"""

In [ ]:
resolve_mistakes(dokumentacja, opcje = ['literowka', 'interpunkcja'], one_shot = True)

 **Instalacja**
Polecenia do instalacji binariów za pomocą Conda lub pip wheels znajdują się na naszej stronie internetowej: https://pytorch.org/get-started/locally/

Jeśli instalujesz ze źródła, będziesz potrzebować:

* Python 3.8 lub nowszy (dla Linuksa wymagany jest Python 3.8.1+)
* Kompilator w pełni, obsługujący C++17, taki jak clang lub gcc (wymagany jest gcc 9.4.0 lub nowszy)
* Zdecydowanie zalecamy zainstalowanie środowiska Anaconda. Otrzymasz wysokiej jakości bibliotekę BLAS (MKL) i kontrolowane wersje zależności niezależnie od dystrybucji Linuksa.

**Obsługa NVIDIA CUDA**
Jeśli chcesz kompilować z obsługą CUDA, wybierz obsługiwaną wersję CUDA z naszej macierzy obsługi, a następnie zainstaluj następujące elementy:

* NVIDIA CUDA
* NVIDIA cuDNN v8.5 lub nowsza
* Kompilator zgodny z CUDA

Uwaga : Możesz zapoznać się z macierzą obsługi cuDNN dla wersji cuDNN z różnymi obsługiwanymi wersjami CUDA, sterownikiem CUDA i sprzętem NVIDIA.

Jeśli chcesz wyłączyć obsługę CUDA, wyeksportu

["[INST] <<SYS>>\nJesteś ekspertem zajmującym się poprawą tesktów w języku polskim.\n<</SYS>>\n\nSprawdź poniższy tekst w języku polskim i popraw go. Zwróć szczególną uwagę na:\n\n* Literówki (pominięcie, powtórzenie lub wstawienie niewłaściwego znaku)\n\n* Błędy interpunkcyjne (zbędne użycie znaku interpunkcyjnego, brak znaku interpunkcyjnego, błędne zapisy w przypadku zbiegu znaków interpunkcyjnych, użycie niewłaściwego znaku interpunkcyjnego)\n\nPoniżej masz przykład: \n\nTeskt do popawy: 'janusz siedział skólnoy za stołm i milczał .'\n\nPrawdiłowy poprawiony teskt: 'Janusz siedział skulony za stołem i milczał.'Pamietaj, aby ingerencja w treść była związana tylko z poprawą błędu i nie ingerowala w sens pierwotnej treści. Poniżej tekst do poprawy:\n\n\n**Instalacja**\nPolecenia do instalacji binariów za pomocą Conda lub pip wheels znajdują się na naszej stronie internetowej: https://pytorch.org/get-started/locally/\n\nJeśli instalujesz ze źródła, będziesz potrzebować:\n\n* Python 3.8

[Tu wnioski po wyniku z Bielik2]

In [ ]:
change_style(dokumentacja, styl = 'publicystyczny')

 Zmień styl napisanego tekstu na formalny.

Pamiętaj, aby ingerencja w treść była związana tylko ze zmianą i nie ingerowała w sens pierwotnej treści. Poniżej tekst do zmiany stylu:

Instalacja
Polecenia do instalacji binariów za pomocą Conda lub pip wheels znajdują się na naszej stronie internetowej: https://pytorch.org/get-started/locally/

Jeśli instalujesz ze źródła, będziesz potrzebować:

* Python 3.8 lub nowszy (dla Linuksa wymagany jest Python 3.8.1+)
* Kompilator w pełni, obsługujący C++17, taki jak clang lub gcc (wymagany jest gcc 9.4.0 lub nowszy)
* Zdecydowanie zalecamy zainstalowanie środowiska Anaconda. Otrzymasz wysokiej jakości bibliotekę BLAS (MKL) i kontrolowane wersje zależności niezależnie od dystrybucji Linuksa.

Obsługa NVIDIA CUDA
Jeśli chcesz kompilować z obsługą CUDA, wybierz obsługiwaną wersję CUDA z naszej macierzy obsługi, a następnie zainstaluj następujące elementy:

* NVIDIA CUDA
* NVIDIA cuDNN v8.5 lub nowsza
* Kompilator zgodny z CUDA
Uwaga : Możesz zapozn

['[INST] <<SYS>>\nJesteś ekspertem zajmującym się zmianą stylu tekstu w języku polskim.\n<</SYS>>\n\nZmień styl napisanego tekstu na \n\nPamietaj, aby ingerencja w treść była związana tylko ze zmianą i nie ingerowala w sens pierwotnej treści. Poniżej tekst do do zmiany stylu:\n\n\n**Instalacja**\nPolecenia do instalacji binariów za pomocą Conda lub pip wheels znajdują się na naszej stronie internetowej: https://pytorch.org/get-started/locally/\n\nJeśli instalujesz ze źródła, będziesz potrzebować:\n\n* Python 3.8 lub nowszy (dla Linuksa wymagany jest Python 3.8.1+)\n* Kompilator w pełni, obsługujący C++17, taki jak clang lub gcc (wymagany jest gcc 9.4.0 lub nowszy)\n* Zdecydowanie zalecamy zainstalowanie środowiska Anaconda. Otrzymasz wysokiej jakości bibliotekę BLAS (MKL) i kontrolowane wersje zależności niezależnie od dystrybucji Linuksa.\n\n**Obsługa NVIDIA CUDA**\nJeśli chcesz kompilować z obsługą CUDA, wybierz obsługiwaną wersję CUDA z naszej macierzy obsługi, a następnie zainstalu

[Tu wnioski po wyniku z Bielik2]

### Podsumowanie
W powyższym notebooku udało nam się stworzyć i przetestować dwie funkcje do poprawy polskiego tekstu: usuwanie błędów językowych oraz zmiana stylu. Jak widzimy model "Bielik" dał sobie radę z tym zdaniem, które mimo że nie jest skomplikowane to dotyczy wielu sytuacji wziętych z życia codziennego i biznesu. 

Aby nieco ostudzić entuzjazm to pamiętajmy, że Bielik to nie oczywiście jeden z wielu dużych modeli językowych które potrafią wykonać takie zadanie, jednak fakt uczenia Bielika w dużo większej liczbie polskich treści sprawia, że model mógł "zobaczyć" więcej przykładów tekstów napisanych poprawnie w ojczystym języku niż ma to miejsce w innych modelach oraz poradzi sobie z dłuższymi tekstami dzięki kontekstowi wielkości 32k tokenów - to więcej niż chociażby gemma-7b czy Llama-3-8B.

Podobne zadania wykonują również inne narzędzia (w szczególności poprawa błędów jak chociaż autokorekty w wielu narzędziach takich jak Microsoft Word oraz różnego rodzaju wtyczki jak Grammarly). Stworzone funkcje jednak mają przewagę, że wymagają jedynie podania tekstu do poprawy i pod jakim kątem chcemy sprawdzić tekst - użytkownik nie musi "przeklikać" poprawek (co oczywiście może nie za sobą również pewne wady). Bielik dodatkowo jest w pełni darmowy - nie trzeba go kupić, ani subskrybować :)

Notebook oczywiście można rozwinąć, udokumentowanie funkcji na pewno pomoże w jej używaniu dla nowych użytkowników a weryfikacja skuteczności tych funkcji na większej liczbie realnych przykładów pozwoli wyciągnąć wnioski na ile tak naprawdę Bielik rozwiązuje przytoczone problemy.

Dziękuje za zapoznanie się z materiałem

*Autor notebooka: Michał Bogacz*

W razie pytań problemów zapraszam do kontaktu [na moim Linkedin](https://www.linkedin.com/in/michalbogacz1/).

To nie jedyny przykład użycia naszego Orła, zapoznaj sie z innymi przykładami jego użycia [na naszym repozytorium](https://github.com/speakleash/Bielik-how-to-start).

Źrodła:
1. 
2. https://github.com/speakleash/Bielik-how-to-start 
3. https://dobryslownik.pl/kompendium-regul-jezykowych/regula/492/ 
4. https://sjp.pwn.pl/poradnia/haslo/Literowka;22233.html
5. https://www.atelier-redakcji.eu/blog/blad-jezykowy/
6. https://contentwriter.pl/style-jezykowe/#32
7. https://github.com/pytorch/pytorch/blob/main/README.md 
8. https://huggingface.co/spaces/speakleash/open_pl_llm_leaderboard
8. https://huggingface.co/speakleash